In [1]:
from img_doc.document import Document, Block
from img_doc.document.extractors.dataset_extractors import dataset2json
from img_doc.document.extractors import TsPubLayNetExtractor
from img_doc.document.data_structures.page.extractors.block_extractors import BaseRandomWalkClassificator

import numpy as np
import json
import random

2024-02-20 16:39:04.139216: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-20 16:39:04.173992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 16:39:04.174030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 16:39:04.175443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 16:39:04.183004: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-20 16:39:04.183543: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dataset_ext = TsPubLayNetExtractor()

In [3]:
rnd_cls = BaseRandomWalkClassificator(properties=["many_dist", "many_angle", "place_in_block"],count_step=50)

def doc2vecs(doc: Document):
    x = []
    y = []
    for page in doc.pages:
        for block in page.blocks:
            x.append(rnd_cls.get_block_vec(block))
            y.append(block.label)
    return x, y

# def doc2vecs(doc: Document):
#     x = []
#     y = []
#     for page in doc.pages:
#         for block1 in page.blocks:
#             for block2 in page.blocks:
#                 x.append(np.concatenate([rnd_cls.get_block_vec(block1), rnd_cls.get_block_vec(block2)]))
#                 y.append([block1.label, block2.label-block1.label])
#     return x, y

In [4]:
test_doc = Document()
test_doc.set_from_path("/home/daniil/program/dataset/mini_ts_publaynet/train/PMC1180429_00004.jpg")
block = Block()
block.segment = test_doc.pages[0].image.segment
block.label = 0
test_doc.pages[0].blocks.append(block)
dataset_ext.extract(test_doc)
vecs, lab = doc2vecs(test_doc)

In [5]:
vec = vecs[0]
len_node_vec = 10
# for i in range(len_node_vec):
    
#     max_ = np.max(np.abs(vec[i::len_node_vec]))
#     vec[i::len_node_vec] = vec[i::len_node_vec]/max_ if max_ != 0 else vec[i::len_node_vec]
#     # print(vec[i::len_node_vec])
vec

array([0.1       , 0.38789008, 0.0942809 , 0.09370426, 0.        ,
       0.89442719, 0.28024989, 1.        , 0.53439153, 0.93706294,
       0.2       , 0.12850588, 0.06285394, 0.09370426, 0.31059715,
       0.9931506 , 0.77298869, 0.99896854, 0.54761905, 0.95524476,
       0.13333333, 0.12850588, 0.21998878, 0.09370426, 0.85669286,
       1.        , 0.55685707, 1.        , 0.54761905, 0.96783217,
       0.2       , 0.12850588, 0.06285394, 0.12493901, 0.11594233,
       1.        , 0.88767962, 1.        , 0.45767196, 0.97062937,
       0.13333333, 1.        , 0.07856742, 0.09370426, 0.98380326,
       0.97513286, 0.58222293, 1.        , 0.44179894, 0.98321678,
       0.13333333, 0.12850588, 0.21998878, 0.09370426, 0.85669286,
       1.        , 0.55685707, 1.        , 0.54761905, 0.96783217,
       0.2       , 0.12850588, 0.06285394, 0.09370426, 0.31059715,
       0.9931506 , 0.77298869, 0.99896854, 0.54761905, 0.95524476,
       0.13333333, 0.12850588, 0.21998878, 0.09370426, 0.85669

In [6]:
dataset = dataset2json("/home/daniil/program/dataset/mini_ts_publaynet/", dataset_ext, doc2vecs)

ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5148785_00000.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5148787_00000.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC3671139_00002.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC3016243_00010.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5215253_00000.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5640724_00000.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5263190_00000.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5640740_00002.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC6060773_00002.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5462463_00023.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC4712356_00003.jpg
ERROR: /home/daniil/program/dataset/mini_ts_publaynet/train/PMC5760672_00005.jpg
ERROR: /home/daniil/program/

In [7]:
print("Первый x:\n", dataset["x"][0])
print("Последний x:\n", dataset["x"][-1])
print("Размер x:\t", len(dataset["x"][0]))
print("Первый y:\t", dataset["y"][0])
print("Последний y:\t", dataset["y"][0])
print("Кол-во записей:\t", len(dataset["x"]))

Первый x:
 [0.05632127 0.02380952 0.24231856 0.41649656 0.74648068 1.
 0.04196876 0.         0.14184397 0.41071429 0.05632127 0.02380952
 0.24231856 0.41649656 0.74648068 1.         0.04196876 0.
 0.14184397 0.41071429 0.11264254 0.0952381  0.02550722 1.
 0.95955965 0.99944491 0.74648068 0.         0.         0.25
 1.         0.0952381  0.05101443 0.99488488 0.         0.99944491
 0.95955965 0.         0.08510638 0.07142857 1.         0.0952381
 0.05101443 0.99488488 0.         0.99944491 0.95955965 0.
 0.08510638 0.07142857 0.54387222 0.0952381  0.05101443 0.11206636
 0.         0.99861783 0.62332578 0.99944491 0.36170213 0.05357143
 1.         0.0952381  0.05101443 0.99488488 0.         0.99944491
 0.95955965 0.         0.08510638 0.07142857 0.54387222 0.0952381
 0.05101443 0.11206636 0.         0.99861783 0.62332578 0.99944491
 0.36170213 0.05357143 0.11264254 0.11904762 0.16579691 0.11206636
 0.62332578 0.79262399 0.44484977 1.         0.43971631 0.25
 0.37781458 0.11904762 0.19130

In [8]:
def shuffle(x_arrays, y_arrays):
    x_vec = np.concatenate(x_arrays)
    y_vec = np.concatenate(y_arrays)
    data = [(x, y) for x, y in zip(x_vec, y_vec)]
    random.shuffle(data)
    x_vec_rnd = np.array([x for x, y in data])
    y_vec_rnd = np.array([y for x, y in data])
    return x_vec_rnd, y_vec_rnd
    

def get_data(dataset, balans=True): #, batch_size=128

    X = np.array(dataset["x"])
    y = np.array(dataset["y"])
    
    keys = np.unique(y)
    Y_keys = dict()
    for i, key in enumerate(keys):
        z = np.zeros(5)
        z[i] = 1
        Y_keys[key] = z
    Y = np.array([Y_keys[yi] for yi in y])

    X_clus_list = []
    Y_clus_list = []

    for i, item in enumerate(Y_keys.keys()):
        X_clus_list.append(X[Y[:,i]==1,:])
        Y_clus_list.append(Y[Y[:,i]==1,:])
        
    if balans:
        balans_len = [len(yc) for yc in Y_clus_list]
        min_len = min(balans_len)
    
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    val_x = []
    val_y = []
    for class_data in zip(X_clus_list, Y_clus_list):
        data = [(x, y) for x, y in zip(class_data[0], class_data[1])]
        random.shuffle(data)
        if balans:
            data = data[:min_len]
        len_data = len(data)
        len_test = round(0.2*len_data)
        len_val = round((len_data-len_test)*0.2)
        len_train = len_data - len_test - len_val
        
        train_x.append([x for x, y in data[:len_train]])
        train_y.append([y for x, y in data[:len_train]])
        
        val_x.append([x for x, y in data[len_train:len_train+len_val]])
        val_y.append([y for x, y in data[len_train:len_train+len_val]])
    
        test_x.append([x for x, y in data[-len_test:]])
        test_y.append([y for x, y in data[-len_test:]])
        
    train_x, train_y = shuffle(train_x, train_y)
    val_x, val_y = shuffle(val_x, val_y)
    test_x, test_y = shuffle(test_x, test_y)
    
    return train_x, train_y, val_x, val_y, test_x, test_y, Y_keys

In [9]:
train_x, train_y, val_x, val_y, test_x, test_y, Y_keys = get_data(dataset, balans=True)

In [10]:
with open("rez.json", "w") as f:
    json.dump({"train_x": train_x.tolist(), "train_y": train_y.tolist(),
               "val_x": val_x.tolist(), "val_y": val_y.tolist(), 
               "test_x": test_x.tolist(), "test_y": test_y.tolist()}, f)

In [11]:
train_x[4]

array([0.12195122, 0.29408585, 0.31220264, 1.        , 1.        ,
       0.99778516, 0.71444757, 0.82530726, 0.71345029, 0.30769231,
       0.41463415, 0.5881717 , 1.        , 0.73730873, 0.71444757,
       0.99896854, 0.        , 1.        , 0.6374269 , 0.94871795,
       0.17073171, 0.5881717 , 0.33763387, 0.65538554, 0.        ,
       0.99705449, 0.        , 0.99896854, 1.        , 1.        ,
       0.47105385, 0.29408585, 0.12855403, 0.32769277, 0.        ,
       1.        , 0.        , 1.        , 1.        , 0.64102564,
       0.14634146, 0.29408585, 0.95107998, 0.32769277, 0.        ,
       1.        , 0.        , 1.        , 0.67836257, 0.64102564,
       0.47105385, 0.29408585, 0.12855403, 0.32769277, 0.        ,
       1.        , 0.        , 1.        , 1.        , 0.64102564,
       0.14634146, 0.29408585, 0.95107998, 0.32769277, 0.        ,
       1.        , 0.        , 1.        , 0.67836257, 0.64102564,
       0.14634146, 0.29408585, 0.95107998, 0.32769277, 0.     

In [12]:
a = np.array([1, 2.3 , 4. , -6])

In [13]:
a/np.max(np.abs(a))

array([ 0.16666667,  0.38333333,  0.66666667, -1.        ])